In [82]:
import cv2
import time
import multiprocessing as mp

In [83]:
# 将逐帧采取到的图片放入队列中
def queue_img_put(q_put, name, pwd, ip, channel=2):
    # 读取摄像头
    cap = cv2.VideoCapture("rtsp://{}:{}@{}/Streaming/Channels/{}".format(name, pwd, ip, channel))
    while True:
        ret, frame = cap.read()
        q_put.put(frame) if ret else None
        q_put.get() if q_put.qsize()>1 else None

In [84]:
# 将逐帧采取到放入队列的图片读取出来
def queue_img_get(q_get, window_name):
    cv2.namedWindow(window_name, flags=cv2.WINDOW_FREERATIO)
    while True:
        frame = q_get.get()
        cv2.imshow(window_name, frame)
        cv2.waitKey(1)

In [85]:
# 多摄像头调用
def run_multi_camera():
    user_name, user_pwd = "admin", "uex_2022"
    camera_ips = [
        "192.168.1.64" # 摄像头1
    ]
    
    mp.set_start_method(method='spawn', force=True)
    queue_camera = [mp.Queue(maxsize=4) for _ in camera_ips]
    
    processes = []
    for queue, camera_ip in zip(queue_camera, camera_ips):
        processes.append(mp.Process(target=queue_img_put, args=(queue, user_name, user_pwd, camera_ip)))
        processes.append(mp.Process(target=queue_img_get, args=(queue, camera_ip)))
        
    for process in processes:
        process.daemon = True
        process.start()
    for process in processes:
        process.join()

In [86]:
 if __name__ == "__main__":
    run_multi_camera()